## Data Exploration

From the task. Three raw datasets have been provided in order to create recommender system which are 

 - user_applications
 - previous_job_post 
 - and active_job_post


In [10]:
import pandas as pd
from IPython.display import display

user = pd.read_csv("user_applications.csv")
previous = pd.read_csv("previous_job_post.csv")
active = pd.read_csv("active_job_post.csv")

display(user.head())
display(previous.head())
display(active.head())


,user_id,job_id
0,1951,7662
1,4517,12970
2,4469,12662
3,502,14356
4,1620,23126


,job_id,job_title,role_name,sub_role_name,location,latitude,longitude,salary,company_name
0,0,Warehouse & Distribution Sales,ขาย,พนักงานขาย/Sales/AE,อ.ศรีราชา จ.ชลบุรี,NaN,NaN,ไม่ระบุ,vueoeo
1,1,เจ้าหน้าที่บรรณาธิการหนังสือเรียน กลุ่มสาระสัง...,นักเขียน/บรรณาธิการ/นักแปลภาษา,บรรณาธิการ,กรุงเทพมหานคร(เขตบางซื่อ),NaN,NaN,ตามตกลง,30jfum
2,2,Education Consultant ประจำสาขา เซ็นทรัลแจ้งวัฒนะ,ขาย,ขายอื่นๆ,นนทบุรี(ปากเกร็ด),NaN,NaN,"15,000-16,000",1d36pp
3,3,★★เริ่มงาน 25 ม.ค.นี้ (ฝ่ายขายประจำOffice รับ...,บริการลูกค้า/ลูกค้าสัมพันธ์/Call Center,บริการ/ลูกค้าสัมพันธ์/Call Center อื่นๆ,กรุงเทพมหานคร(ทุกเขต),NaN,NaN,"13,000-50,000",wqalnh
4,4,ผู้จัดการอาคาร (ใกล้ ม.ธรรมศาสตร์ รังสิต),ผู้จัดการ/ผู้อำนวยการ/MD/CEO,ผู้จัดการทั่วไป,อ.คลองหลวง จ.ปทุมธานี,14.064528,100.609091,"28,000 - 33,000",p1he99


,job_id,job_title,role_name,sub_role_name,location,latitude,longitude,salary,company_name
0,25320,พนักงานขับรถรับส่งพนักงาน,คนขับรถ,คนขับรถ อื่นๆ,"กรุงเทพมหานคร(เขตพระโขนง),กรุงเทพมหานคร(เขตสวน...",NaN,NaN,ตามตกลง,83akmq
1,25321,ผู้ประเมินราคางานตกแต่งภายใน,โยธา/สำรวจ/สถาปัตย์/มัณฑนากร/ประเมินราคา,ประเมินราคา,อ.เมืองนนทบุรี จ.นนทบุรี,13.872826,100.53001,"25,000-35,000 ขึ้นอยู่กับประสบการณ์",b57auw
2,25322,นักออกแบบกราฟิก (Graphic design),ศิลปะ/กราฟฟิค/ออกแบบ/ช่างภาพ,Graphic Design/สิ่งพิมพ์,เขตคลองสาน กรุงเทพมหานคร,NaN,NaN,"18,000-25,000 บาท",362uhk
3,25323,พนักงานขับรถส่งสินค้า ประจำศูนย์กระจายสินค้า ส...,คนขับรถ,คนขับรถส่งสินค้า,กรุงเทพมหานคร(เขตคันนายาว),NaN,NaN,"16,000-18,000",yp1i9y
4,25324,พนักงานขายทางโทรศัพท์​ / Telesales,ขาย,พนักงานขายทางโทรศัพท์/Telesales,เขตปทุมวัน กรุงเทพมหานคร,NaN,NaN,"15,000 + คอมมิสชั่น",4s96ty


In [13]:
#Exploration for columns and values

print("-----user application------")
display(user.dtypes)

print("-----previous job post------")
display(previous.dtypes)

print("-----active job post------")
display(active.dtypes)


-----user application------


user_id    int64
job_id     int64
dtype: object

-----previous job post------


job_id             int64
job_title         object
role_name         object
sub_role_name     object
location          object
latitude         float64
longitude        float64
salary            object
company_name      object
dtype: object

-----active job post------


job_id             int64
job_title         object
role_name         object
sub_role_name     object
location          object
latitude         float64
longitude        float64
salary            object
company_name      object
dtype: object

## Data Cleaning and feature selection

From data exploration, I choose 4 attributes to use in a feature space as following;

- role_name -> role_name, unlike job_title, can be easily clean and turn into catagorical dataset. This is because role_name contain short and meaningful words each row.
- sub_role_name -> the same reason as role_name
- location -> It will not be used directly. Instead, this attribute could be extracted and use their information which is province. 
- company_name -> company_name can be easily clean and turn into catagorical dataset. It usually is one of the reason for job application.

Firstly, all attributes have to translate into English. This is because English language is easy for data cleaning than Thai langauge (Complexity of Language, Library for NLP Application, etc.). Google translator libs is called for this work.


In [14]:
from google_trans_new import google_translator

def translation(df):
    translator = google_translator()
    unique_elements = df.unique()
    translations = {}

    for element in sorted(unique_elements):
        # add translation to the dictionary
        translation = translator.translate(element, lang_tgt='en')
        print(element + " -> " + translation)
        translations[element] = translation

    print(translations)
    return translations


In [15]:
active.dropna(subset=['role_name','sub_role_name'], inplace=True)
previous.dropna(subset=['role_name','sub_role_name'], inplace=True)

active_translations = translation(active['role_name'])
previous_translations = translation(previous['role_name'])

active['role_name'].replace(active_translations, inplace=True)
previous['role_name'].replace(previous_translations, inplace=True)

display(active.head())
display(previous.head())

E-commerce -> E-commerce 
Freelance -> Freelance 
Part-time/พนักงานสัญญาจ้าง -> Part-time employment contract 
กฎหมาย -> law 
การตลาด -> Marketing 
การตลาด / PR -> Marketing / PR 
การผลิต / QA&QC -> Production / QA & QC 
การศึกษา/ วิชาการ -> Education / Academic 
การแพทย์ -> Medical 
ก่อสร้าง -> Construction 
ขนส่ง/คลังสินค้า -> Transport / Warehouse 
ขนส่งและคลังสินค้า -> Transport and warehouse 
ขับรถ -> Drive 
ขาย -> sell 
คนขับรถ -> Drivers 
คลังสินค้า -> warehouse 
ความสะอาดและสวน -> Cleanliness and garden 
คอมพิวเตอร์/IT/โปรแกรมเมอร์ -> Computer / IT / Programmer 
งาน Part-time/พนักงานชั่วคราว -> Part-time work / temporary employees 
งานการตลาด งาน PR -> Marketing Jobs PR 
งานการผลิต -> Production work 
งานการเงิน-ธนาคาร -> Finance - Bank 
งานขนส่ง -> Transportation 
งานขนส่ง-คลังสินค้า -> Transportation - Warehouse 
งานขาย งานบริการลูกค้า งานพัฒนาธุรกิจ -> Customer service sales Business development 
งานจัดซื้อ -> Purchasing 
งานธนาคาร งานการเงิน -> Banking Banking 
งานธุรการ งา

Freelance -> Freelance 
Part-time/พนักงานสัญญาจ้าง -> Part-time employment contract 
กฎหมาย -> law 
การตลาด -> Marketing 
การตลาด / PR -> Marketing / PR 
การผลิต / QA&QC -> Production / QA & QC 
การศึกษา/ วิชาการ -> Education / Academic 
การแพทย์ -> Medical 
ก่อสร้าง -> Construction 
ก่อสร้าง โยธา และสถาปัตยกรรม -> Civil and architecture construction 
ขนส่ง/คลังสินค้า -> Transport / Warehouse 
ขนส่งและคลังสินค้า -> Transport and warehouse 
ขับรถ -> Drive 
ขาย -> sell 
คนขับรถ -> Drivers 
คลังสินค้า -> warehouse 
คอมพิวเตอร์/IT/โปรแกรมเมอร์ -> Computer / IT / Programmer 
งาน Part-time/พนักงานชั่วคราว -> Part-time work / temporary employees 
งานการตลาด งาน PR -> Marketing Jobs PR 
งานการผลิต -> Production work 
งานการเงิน-ธนาคาร -> Finance - Bank 
งานขนส่ง -> Transportation 
งานขนส่ง-คลังสินค้า -> Transportation - Warehouse 
งานขาย งานบริการลูกค้า งานพัฒนาธุรกิจ -> Customer service sales Business development 
งานจัดซื้อ -> Purchasing 
งานธนาคาร งานการเงิน -> Banking Banking 
งานธุรการ งา

,job_id,job_title,role_name,sub_role_name,location,latitude,longitude,salary,company_name
0,25320,พนักงานขับรถรับส่งพนักงาน,Drivers,คนขับรถ อื่นๆ,"กรุงเทพมหานคร(เขตพระโขนง),กรุงเทพมหานคร(เขตสวน...",NaN,NaN,ตามตกลง,83akmq
1,25321,ผู้ประเมินราคางานตกแต่งภายใน,Civil / Survey / Architecture / Decorative / A...,ประเมินราคา,อ.เมืองนนทบุรี จ.นนทบุรี,13.872826,100.53001,"25,000-35,000 ขึ้นอยู่กับประสบการณ์",b57auw
2,25322,นักออกแบบกราฟิก (Graphic design),Art / Graphic / Design / Photographer,Graphic Design/สิ่งพิมพ์,เขตคลองสาน กรุงเทพมหานคร,NaN,NaN,"18,000-25,000 บาท",362uhk
3,25323,พนักงานขับรถส่งสินค้า ประจำศูนย์กระจายสินค้า ส...,Drivers,คนขับรถส่งสินค้า,กรุงเทพมหานคร(เขตคันนายาว),NaN,NaN,"16,000-18,000",yp1i9y
4,25324,พนักงานขายทางโทรศัพท์​ / Telesales,sell,พนักงานขายทางโทรศัพท์/Telesales,เขตปทุมวัน กรุงเทพมหานคร,NaN,NaN,"15,000 + คอมมิสชั่น",4s96ty


,job_id,job_title,role_name,sub_role_name,location,latitude,longitude,salary,company_name
0,0,Warehouse & Distribution Sales,sell,พนักงานขาย/Sales/AE,อ.ศรีราชา จ.ชลบุรี,NaN,NaN,ไม่ระบุ,vueoeo
1,1,เจ้าหน้าที่บรรณาธิการหนังสือเรียน กลุ่มสาระสัง...,Writer / Editor / Translator,บรรณาธิการ,กรุงเทพมหานคร(เขตบางซื่อ),NaN,NaN,ตามตกลง,30jfum
2,2,Education Consultant ประจำสาขา เซ็นทรัลแจ้งวัฒนะ,sell,ขายอื่นๆ,นนทบุรี(ปากเกร็ด),NaN,NaN,"15,000-16,000",1d36pp
3,3,★★เริ่มงาน 25 ม.ค.นี้ (ฝ่ายขายประจำOffice รับ...,Customer Service / Customer Relations / Call C...,บริการ/ลูกค้าสัมพันธ์/Call Center อื่นๆ,กรุงเทพมหานคร(ทุกเขต),NaN,NaN,"13,000-50,000",wqalnh
4,4,ผู้จัดการอาคาร (ใกล้ ม.ธรรมศาสตร์ รังสิต),Manager / Director / MD / CEO,ผู้จัดการทั่วไป,อ.คลองหลวง จ.ปทุมธานี,14.064528,100.609091,"28,000 - 33,000",p1he99


In [19]:
active_sub_translations = translation(active['sub_role_name'])
previous_sub_translations = translation(previous['sub_role_name'])

active['sub_role_name'].replace(active_sub_translations, inplace=True)
previous['sub_role_name'].replace(previous_sub_translations, inplace=True)

Advertising/Creative -> Advertising/Creative 
Application   Support/ Helpdesk -> Application   Support/ Helpdesk 
Application Network -> Application Network 
BOI -> Book 
Back End Developer -> Back End Developer 
Bancassurance -> Bancassurance 
Brand Marketing -> Brand Marketing 
Brand/Product Marketing -> Brand/Product Marketing 
Business Analyst (BA) -> Business Analyst (ba) 
Business Analyst(BA) -> Business Analyst (ba) 
Call Center -> Call Center 
Computer Graphic/3D/Animation -> Computer Graphic/3D/Animation 
Copy writer -> Copy writer 
Credit Control -> Credit Control 
Data Engineer/Big Data -> Data Engineer/Big Data 
Data Scientist -> Data Scientist 
Database Administration -> Database Administration 
DevOps Engineer -> DevOps Engineer 
Digital Marketing -> Digital Marketing 
E-commerce อื่นๆ -> Other e-commerce 
Full Stack Developer -> Full Stack Developer 
Graphic Design/สิ่งพิมพ์ -> GRAPHIC DESIGN / Paper 
Hardware -> Hardware 
IT Audit -> IT Audit 
IT Manager -> IT Manager 


ตรวจสอบอาคาร/สำรวจ -> Building inspection / exploration 
ตรวจสินค้า -> Check the product 
ตัวแทนประกันภัย -> Insurance agent 
ต้อนรับลูกค้า -> Welcome customers 
ทนายความ -> lawyer 
ทันตแพทย์ -> dentist 
ทัวร์/บริษัททัวร์ -> Tour / Tour Company 
ที่ปรึกษาการขาย -> Sales consultant 
ที่ปรึกษาด้านกฎหมาย -> Legal consultant 
ที่ปรึกษาทางการเงิน -> Financial advisor 
ที่ปรึกษาอสังหาริมทรัพย์ -> Real estate consultant 
ที่ปรึกษาไอที -> IT consultant 
ท่องเที่ยวอื่นๆ -> Other travel 
ธนาคาร/การเงินอื่นๆ -> Bank / Other Finance 
ธรณีวิทยาและป่าไม้อื่นๆ -> Other geology and forests 
ธุรการ/การจัดการทั่วไป -> Administration / General management 
ธุรการทั่วไป -> General Administration 
ธุรการบัญชี -> Accounting Administration 
ธุรการฝ่ายขาย -> Sales Administration 
ธุรการอื่นๆ/การจัดการทั่วไปอื่นๆ -> Other administrative / other general management 
ธุรกิจการบิน -> Aviation business 
นักกฎหมาย -> Lawyer 
นักกายภาพบำบัด -> Physical therapist 
นักข่าว -> journalist 
นักคณิตศาสตร์/สถิติ -> Mathemati

วิศวกรออกแบบ -> Design engineer 
วิศวกรอุตสาหการ/โรงงาน -> Industrial Engineer / Factory 
วิศวกรเขียนแบบ -> Write engineer 
วิศวกรเคมี/ปิโตรเลียม -> Chemical / Petroleum Engineer 
วิศวกรเครื่องกล -> Mechanical engineer 
วิศวกรเครื่องกล/ยานยนต์ -> Mechanical / Automotive Engineer 
วิศวกรแมคคาทรอนิกส์ -> Machatronics Engineer 
วิศวกรโครงการ -> Project Engineer 
วิศวกรโทรคมนาคม -> Telecommunications engineer 
วิศวกรโยธา -> Civil Engineer 
วิศวกรโยธา/ก่อสร้าง -> Civil / Construction Engineer 
วิศวกรไฟฟ้า/วิศวกรอิเล็กทรอนิกส์ -> Electrical Engineer / Electronics Engineer 
วิศวกรไฟฟ้า/อิเลคโทรนิค/สื่อสาร -> Electrical / Electronic Engineer / Communication 
วิศวะอื่นๆ -> Other engineering 
วิเคราะห์การลงทุน -> Investment analysis 
วิเคราะห์ข้อมูลทั่วไป -> General information analysis 
วิเคราะห์สถิติ -> Statistical analysis 
วิเคราะห์สินเชื่อ/อนุมัติสินเชื่อ -> Credit analysis / Credit approval 
ว่าจ้างและเงินเดือน -> Hiring and salary 
สถาปนิก -> architect 
สถาปนิก / มัณฑนากร อื่นๆ -> Archite

Advertising/Creative -> Advertising/Creative 
Application   Support/ Helpdesk -> Application   Support/ Helpdesk 
Application Network -> Application Network 
BOI -> Book 
Back End Developer -> Back End Developer 
Bancassurance -> Bancassurance 
Brand Marketing -> Brand Marketing 
Brand/Product Marketing -> Brand/Product Marketing 
Business Analyst (BA) -> Business Analyst (ba) 
Business Analyst(BA) -> Business Analyst (ba) 
Call Center -> Call Center 
Computer Graphic/3D/Animation -> Computer Graphic/3D/Animation 
Credit Control -> Credit Control 
Data Engineer/Big Data -> Data Engineer/Big Data 
Data Scientist -> Data Scientist 
Database Administration -> Database Administration 
Digital Marketing -> Digital Marketing 
E-commerce อื่นๆ -> Other e-commerce 
Front End Web Developer -> Front End Web Developer 
Graphic Design/สิ่งพิมพ์ -> GRAPHIC DESIGN / Paper 
Hardware -> Hardware 
IT Audit -> IT Audit 
IT Manager -> IT Manager 
IT Manager/Senior Programmer -> IT Manager/Senior Programm

ที่ปรึกษาทางการเงิน -> Financial advisor 
ที่ปรึกษาอสังหาริมทรัพย์ -> Real estate consultant 
ที่ปรึกษาไอที -> IT consultant 
ท่องเที่ยวอื่นๆ -> Other travel 
ธนาคาร/การเงินอื่นๆ -> Bank / Other Finance 
ธรณีวิทยาและป่าไม้อื่นๆ -> Other geology and forests 
ธุรการ/การจัดการทั่วไป -> Administration / General management 
ธุรการทั่วไป -> General Administration 
ธุรการบัญชี -> Accounting Administration 
ธุรการฝ่ายขาย -> Sales Administration 
ธุรการอื่นๆ/การจัดการทั่วไปอื่นๆ -> Other administrative / other general management 
ธุรกิจการบิน -> Aviation business 
นักกฎหมาย -> Lawyer 
นักกายภาพบำบัด -> Physical therapist 
นักข่าว -> journalist 
นักคณิตศาสตร์/สถิติ -> Mathematician / Statistics 
นักปฐพี -> Gate 
นักวิจัยและพัฒนาผลิตภัณฑ์ -> Product research and development 
นักวิทยาศาสตร์/เคมี/ชีวะ/ฟิสิกส์ -> Scientists / Chemistry / Chemical / Physics 
นักวิเคราะห์/เศรษฐศาสตร์ อื่นๆ -> Analyst / other economics 
นักศึกษาจบใหม่ -> New graduate student 
นักศึกษาฝึกงาน -> Internship students 
นักส

วิศวกรแมคคาทรอนิกส์ -> Machatronics Engineer 
วิศวกรโครงการ -> Project Engineer 
วิศวกรโทรคมนาคม -> Telecommunications engineer 
วิศวกรโยธา -> Civil Engineer 
วิศวกรโยธา/ก่อสร้าง -> Civil / Construction Engineer 
วิศวกรโลหะการ -> Metal engineers 
วิศวกรไฟฟ้า/วิศวกรอิเล็กทรอนิกส์ -> Electrical Engineer / Electronics Engineer 
วิศวกรไฟฟ้า/อิเลคโทรนิค/สื่อสาร -> Electrical / Electronic Engineer / Communication 
วิศวะอื่นๆ -> Other engineering 
วิเคราะห์การลงทุน -> Investment analysis 
วิเคราะห์ข้อมูลทั่วไป -> General information analysis 
วิเคราะห์สถิติ -> Statistical analysis 
วิเคราะห์สินเชื่อ/อนุมัติสินเชื่อ -> Credit analysis / Credit approval 
ว่าจ้างและเงินเดือน -> Hiring and salary 
สถาปนิก -> architect 
สถาปนิก / มัณฑนากร อื่นๆ -> Architects / Other decorations 
สถาปัตยกรรม -> architecture 
สถาปัตยกรรมภายในและมัณฑนศิลป์ -> Interior architecture and decorative arts 
สปาบำบัด/ฟิตเนส/กีฬา/สันทนาการ -> Spa Therapy / Fitness / Sport / Recreation 
สรรหาบุคลากร -> Recruit 
สหกิจศึกษา -> 

&ensp;

For the 'province' column, the 'ThaiAddressParser' library is applied to extract location information. This is also effortless to translate in to English language.


In [22]:
import ThaiAddressParser

def FindProvince(df):
    provinces = []
    for row in df:
        try:
            provinces.append(ThaiAddressParser.parse(row)['province']['en'])
        except:
            provinces.append('')

    return provinces

In [25]:
active = active.dropna(subset=['location'])
previous = previous.dropna(subset=['location'])


previous['location'] = previous['location'].astype('str')
active['location'] = active['location'].astype('str')

previous['province'] = FindProvince(previous['location'])
active['province'] = FindProvince(active['location'])

previous = previous.dropna(subset=['province'])
active = active.dropna(subset=['province'])


In [17]:
def clean_role_name(df):
    df = df.str.split("/", n = 1, expand = True)[0]
    df = df.str.lower()
    df = df.str.strip()
    return df

In [26]:
all_data = pd.concat([previous, active])

all_data.dropna(subset=['role_name', 'sub_role_name', 'company_name', 'province'], inplace=True)
all_data = all_data[~all_data.province.str.contains("Nan")]

all_data['role_name'] = clean_role_name(all_data['role_name'])

all_data['sub_role_name'] = clean_role_name(all_data['sub_role_name'])


&ensp;

After cleaning white spaces, lowering character cases, and selecting only first element from name with '/', it is matching process. Some role names may have same position or job but they spelled differently such as 'account' and 'accounting'. To tackle down this problem, 'fuzzywuzzy' library is implemented.

## FuzzyWuzzy

FuzzyWuzzy is a library of Python which is used for string matching. Fuzzy string matching is the process of finding strings that match a given pattern. Basically it uses Levenshtein Distance to calculate the differences between sequences.

In [28]:
from fuzzywuzzy import process

def matching_process(df):
    role_dict = {}
    roles = sorted(df.unique())
    for n,role in enumerate(roles):
        print(str(n) +" " + role)
        if len(role)>= 5:
            matches = process.extract(role, df.unique(), limit = len(df.unique()))
            for match in matches:
              # Check whether the similarity score is greater than or equal to 80
              if match[1] >= 90 and len(match[0])>=5 and match[1] < 100:
                if len(match[0]) < len(role):
                    print(match[0] + " -> " + role + ": " +str(match[1]))
                    role_dict[match[0]] = role
                else:
                    print(role + " -> " + match[0] + ": " + str(match[1]))
                    role_dict[role] = match[0]
    print(role_dict)
    return (role_dict)

In [31]:

roles = matching_process(all_data['role_name'])


# There are sensitive case that have to manually eliminate as fuzzywuzzy may not correctly match in some cases.

del roles["administration"]
del roles["design"]

roles['warehouse'] = 'transportation - warehouse'
roles['transport and warehouse'] = 'transportation - warehouse'
roles['banking banking '] = 'bank'

all_data['role_name'] = all_data['role_name'].replace(roles)
all_data['sub_role_name'] = all_data['sub_role_name'].replace(roles)

0 account
1 accounting
2 administration
administration -> human resources administration: 90
3 advertising
4 analyst
5 architect
6 art
7 aviation business
8 bank
9 banking banking
10 beauty service
11 business development and organization development
12 civil
13 cleanliness and garden
14 clothing
15 computer
16 construction
17 customer service
customer service -> customer service-call center: 90
customer service -> customer service sales business development: 90
18 customer service sales business development
customer service -> customer service sales business development: 90
19 customer service-call center
customer service -> customer service-call center: 90
20 debt staff
21 department store
22 design
design -> website design: 90
23 doctors
24 drawing
25 drive
26 drivers
27 e-commerce
ecommerce -> e-commerce: 95
28 ecommerce
ecommerce -> e-commerce: 95
29 education
30 engineer
31 engineering
32 entertainment
33 factory
34 finance - bank
35 food
36 freelance
37 gems and jewelry
38 geogr

In [ ]:
#for later use of the role dictionary, contain  it as pickle file

import pickle

a_file = open("roles.pkl", "wb")
pickle.dump(roles, a_file)
a_file.close()


## LabelEndcoder

After translation and matching, it is the label endcoding process. In order to build model, features may have to be endcoded. This method may not be very crucial since K-mode can easily deal with catagorical dataset. On the other hand, endcoding feature could possibly enchance the model or reduce calculation time. In addtion to echancement, labelling may help us for implementation in others algorithms.

To use LabelEndcoder, we have to import the library from scikit-learn. Merging active and previous tables together is required as the model need to learn all possible data.

In [18]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
df_label = pd.DataFrame(all_data, columns=['role_name','sub_role_name','province','company_name'])

lab_enc = LabelEncoder()
for column in df_label.columns:
    df_label[column] = lab_enc.fit_transform(df_label[column])
    pkl_file = column + "_LabelEncoder.pkl"
    with open(pkl_file, 'wb') as file:
        pickle.dump(lab_enc, file)

## Modeling (KModes clustering)

Everyone may heard about K-mean clustering. It is one of the most famous methods for unsupervised learning, specifically clustring model. However K-mean applies Euclidean distance for the purpose of creating clusters which is not quite effective for categorical data likes these dataset.

Thus, the model should implement K-Modes clustering. This method measures similarity of categorical data and build clusters. The algorithm is useful and its library is provided in Python. 

### Training Model on Previous Job Post

In [ ]:
previous['role_name'] = clean_role_name(previous['role_name'])

previous['sub_role_name'] = clean_role_name(previous['sub_role_name'])

a_file = open("roles.pkl", "rb")
roles = pickle.load(a_file)

previous['role_name'] = previous['role_name'].replace(roles)
previous['sub_role_name'] = previous['sub_role_name'].replace(roles)

print(previous.head(10))
#
previous_endc = pd.DataFrame(previous, columns=['role_name','sub_role_name','province','company_name'])

lab_enc = LabelEncoder()

for column in previous_endc.columns:
    pkl_file = column + "_LabelEncoder.pkl"
    with open(pkl_file, 'rb') as file:
        pickle_model = pickle.load(file)
    previous_endc[column] = pickle_model.transform(previous_endc[column])
    
km = KModes(n_clusters=80, init='Huang', n_init=5, verbose=1).fit(df)

kmodes_model = "job_kmodes_model.pkl"
with open(kmodes_model, 'wb') as file:
    pickle.dump(km, file)

y = km.predict(df)

previous['cluster'] = y

### Prediction on Active Job Post

In [ ]:
active['role_name'] = clean_role_name(active['role_name'])

active['sub_role_name'] = clean_role_name(active['sub_role_name'])

a_file = open("roles.pkl", "rb")
roles = pickle.load(a_file)

active['role_name'] = active['role_name'].replace(roles)
active['sub_role_name'] = active['sub_role_name'].replace(roles)

print(active.groupby('role_name')['job_id'].nunique())

active_enc = pd.DataFrame(active, columns=['role_name','sub_role_name','province','company_name'])

lab_enc = LabelEncoder()
for column in active_enc.columns:
    pkl_file = column + "_LabelEncoder.pkl"
    with open(pkl_file, 'rb') as file:
        pickle_model = pickle.load(file)
    active_enc[column] = pickle_model.transform(active_enc[column])

kmodes_model = "job_kmodes_model.pkl"
with open(kmodes_model, 'rb') as file:
    km = pickle.load(file)

y = km.predict(df)

active['cluster'] = y

## Clustering Model Evaluation using Elbow curve 

For evaluation part, Elbow curve method is implemented. In cluster analysis, the elbow method is a heuristic used in determining the number of clusters in a data set. The method consists of plotting the explained variation as a function of the number of clusters, and picking the elbow of the curve as the number of clusters to use. The same method can be used to choose the number of parameters in other data-driven models, such as the number of principal components to describe a data set.

In [ ]:
################################## Evaluation Part ############################################

import matplotlib.pyplot as plt

sse = []
for k in range(1,150):
    print('k = '+ str(k))
    kmodes = KModes(n_clusters=k)
    kmodes.fit(df)
    sse.append(kmodes.cost_)

plt.style.use("fivethirtyeight")
plt.plot(range(1,150),sse)
plt.xticks(range(1,150))
plt.xlabel("Clusters")
plt.ylabel("SSE")
plt.show()

![title](img/EvaluateElbow.png)

## Generating Recommendation

In [ ]:
previous_cluster = previous[['job_id','role_name','sub_role_name','province','company_name']]
active_cluster = active[['job_id','role_name','sub_role_name','province','company_name']]

df = pd.merge(user,previous_cluster,how="left",left_on='job_id',right_on='job_id')

df.dropna(subset=['cluster'],inplace=True)
df['recommendation'] = ""

## Gower distance

Gower’s Distance can be used to measure how different two records are. The records may contain combination of logical, categorical, numerical or text data. The distance is always a number between 0 (identical) and 1 (maximally dissimilar). The metrics used for each data type are described below:

- quantitative (interval): range-normalized Manhattan distance
- ordinal: variable is first ranked, then Manhattan distance is used with a special adjustment for ties
- nominal: variables of k categories are first converted into k binary columns and then the Dice coefficient is used


In [ ]:
import gower

for ind in df.index:

    searching_previous = previous_cluster.loc[previous_cluster.job_id == df.job_id[ind]]
    searching_active = active_cluster.loc[active_cluster.cluster == df.cluster[ind]]

    searching = pd.concat([searching_previous,searching_active])
    first_row = gower.gower_matrix(searching)[0]
    values = []
    try:
        first_min = min(first_row[first_row != min(first_row)])
        second_min = min(first_row[(first_row != first_min) & (first_row != min(first_row))])
        res = searching[first_row == first_min]
        res2 = searching[first_row == second_min]
        values = list(res.job_id.values) + list(res2.job_id.values)
        print(values)

    except:
        try:
            samp_value = active_cluster['job_id'].loc[active_cluster['cluster'] == int(df['cluster'][ind])].sample(n=2,random_state=1,replace=True).values
            values = values + list(set(samp_value) - set(values))
        except:
            samp_value = active_cluster['job_id'].loc[active_cluster['cluster'] == int(df['cluster'][ind])].values
            values = values + list(set(samp_value) - set(values))
        print("no similar Job")
    print(values)
    df['recommendation'][ind] = values

recommendation = df[['user_id','recommendation']]
recommendation['recommendation'] = recommendation['recommendation'].str.strip('[]')
res = recommendation.groupby('user_id').agg({'recommendation': lambda x: ','.join(x) })

## Result from Recommendation 


In [36]:

results = pd.read_csv("Result.csv")

display(results.head())

,user_id,recommendation
0,0,"54623, 25341,25351, 25467,27049, 27409"
1,1,"25792, 27184,30067, 30984,36584, 25439,26060, ..."
2,3,"25725, 26981,25368, 27051,25356, 25395,25372, ..."
3,4,"25430, 25800,27604, 30168,34247, 39366,28435, ..."
4,5,"25783, 27383,25756, 26196,25416, 25446"


In [42]:
job_rec = [54623, 25341]

display(active.loc[active['job_id'].isin(job_rec)])

job_rec2 = [25792,27184]

display(active.loc[active['job_id'].isin(job_rec2)])


,job_id,job_title,role_name,sub_role_name,location,latitude,longitude,salary,company_name,province
21,25341,พนักงานจัดเรียงสินค้า(Merchandise วิ่ง Big C บ...,sell,Staff in the shop / PC,กรุงเทพมหานคร(เขตบางนา),NaN,NaN,"10,500-12,000",oiexhh,Bangkok
29303,54623,พนักงานขาย ประจำร้าน Loft @ ศูนย์การค้าพาราไดซ...,sell,Staff in the shop / PC,เขตปทุมวัน กรุงเทพมหานคร,13.746739,100.531893,เงินเดือน + Commission + โบนัส + สวัสดิการอื่นๆ,7p93l6,Bangkok


,job_id,job_title,role_name,sub_role_name,location,latitude,longitude,salary,company_name,province
472,25792,พนักงานขาย (ประจำโลตัสพระราม 2),sell,retail,"กรุงเทพมหานคร(เขตบางขุนเทียน),กรุงเทพมหานคร(เข...",NaN,NaN,"10,000",89smlj,Bangkok
1864,27184,"**พนักงานแนะนำสินค้า บ.ทรู (ซิมเติมเงิน,แพ็คเก...",sell,retail,"กรุงเทพมหานคร(เขตห้วยขวาง),กรุงเทพมหานคร(เขตดิ...",NaN,NaN,"12,500-15,000",hrqhp2,Bangkok


## Flaws of the system and Future works


<font size="3">Of course, the system is not perfect. In stead, there are a lot of gap that may be fixed or looked after.
&nbsp;
 - first of all, the system prone to new kinds of career. If there is a new career that can not match to role dictionary, it will cause a problem. This can be fixed be checking and updating new career to the label encoder model and role dictionary.
&nbsp;
 - Clustering lacking of evaluation and could add more features. Adding more features will increase recommendation efficiency and possibility to deliver to right target. 
&nbsp;
 - Finding matching by the Gower distance method takes long time to calculate. Although clustered datasets are built for this purpose, it still takes a lot of time to process. Threading and load balancing are suggested for tackling down this problem. 
</font>